In [6]:
!pip install streamlit --force-reinstall --upgrade
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.2/208.2 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.2/323.2 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
^C


In [1]:
import streamlit as st
from docx import Document
import re
import io
import time
import csv
import  pandas as pd
from datetime import datetime

In [2]:
import re
import os
from docx import Document

def extract_blanks_with_context(file_path):
    doc = Document(file_path)
    placeholders = []
    for para in doc.paragraphs:
        full_context = para.text.strip()
        if not full_context:
            continue

        # Split text into chunks around blanks
        parts = re.split(r"(_{3,}|20__+)", full_context)

        for i, part in enumerate(parts):
            if re.fullmatch(r"_{3,}|20__+", part):
                # Surrounding words: previous + blank + next
                before = parts[i-1].strip() if i > 0 else ""
                after = parts[i+1].strip() if i+1 < len(parts) else ""
                snippet = f"{before} {part} {after}".strip()

                placeholders.append({
                    "context": full_context,          # full paragraph
                    "doc": os.path.basename(file_path),
                    "text": snippet,                  # phrase with the blank
                    "label": ""
                })
    return placeholders

In [3]:
train_doc_path = [
    "https://storage.googleapis.com/legal_docs_scribd/367987830-Affidavit-of-Service.docx",
    "https://storage.googleapis.com/legal_docs_scribd/65476676-Rental-Contract.docx",
    "https://storage.googleapis.com/legal_docs_scribd/Power_Of_Attorney.docx",
    "https://storage.googleapis.com/legal_docs_scribd/TRAINING_AGREEMENT.docx",
    "https://storage.googleapis.com/legal_docs_scribd/lease_doc.docx"
]

In [4]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=8119f425d7c6944cec6de2408049eb995841f845e1ed3a93272d286144b17055
  Stored in directory: /root/.cache/pip/wheels/01/46/3b/e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget


In [5]:
import wget
import os

In [6]:
def download_file(url, out):
    wget.download(url, out = out)
downloaded_files = []
for url in train_doc_path:
    print(f"Attempting to download: {url}")
    outpath = os.path.basename(url)
    download_file(url, outpath)
    downloaded_files.append(outpath)

Attempting to download: https://storage.googleapis.com/legal_docs_scribd/367987830-Affidavit-of-Service.docx
Attempting to download: https://storage.googleapis.com/legal_docs_scribd/65476676-Rental-Contract.docx
Attempting to download: https://storage.googleapis.com/legal_docs_scribd/Power_Of_Attorney.docx
Attempting to download: https://storage.googleapis.com/legal_docs_scribd/TRAINING_AGREEMENT.docx
Attempting to download: https://storage.googleapis.com/legal_docs_scribd/lease_doc.docx


In [7]:
all_blanks = []
for path in downloaded_files:
    all_blanks.extend(extract_blanks_with_context(path))

In [8]:
def export_to_csv(all_blanks):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"placeholders_{timestamp}.csv"

    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["context", "doc", "text", "label"]) # Added 'context' to fieldnames
        writer.writeheader()
        writer.writerows(all_blanks)

    print(f"Saved {len(all_blanks)} placeholders to {filename}")

In [9]:
export_to_csv(all_blanks)

Saved 114 placeholders to placeholders_20251103_035144.csv


In [10]:
download_file("https://storage.googleapis.com/legal_docs_scribd/placeholders_20251102_002408.csv","placeholder.csv")

In [11]:
import pandas as pd

# Load data
df = pd.read_csv("placeholder.csv")

# Function to infer generalized field type
def infer_type_from_label(label: str) -> str:
    label = str(label).lower()
    if any(kw in label for kw in ["date", "day", "month", "year"]):
        return "date"
    elif any(kw in label for kw in ["amount", "rent", "deposit", "fee"]):
        return "amount"
    elif "signature" in label:
        return "signature"
    elif "checkbox" in label:
        return "checkbox"
    elif "address" in label:
        return "address"
    elif "email" in label:
        return "email"
    elif "phone" in label:
        return "phone"
    return "text"

# Apply mapping directly to DataFrame
df["field_type"] = df["label"].apply(infer_type_from_label)

# Build mapping dictionary (unique labels → field types)
label_type_map = df.dropna(subset=["label"]).drop_duplicates("label") \
                   .set_index("label")["field_type"].to_dict()

# Display mapping
print("Label → Field Type Mapping:")
for label, field_type in label_type_map.items():
    print(f"{label}: {field_type}")

Label → Field Type Mapping:
RegistryNumber: text
DcumentNumber: text
PageNumber: text
BookNumber: text
Date: date
Name: text
Occupants: text
FreeText: text
Address: address
StartDate, EndDate: date
RentDueDate: date
MonthlyRent: date
DepositAmount: amount
AdultCount, ChildCount: text
KeyCount: text
PrincipalName: text
PrincipalAddress: address
PrincipalCity: text
PrincipalState: text
AttorneyName: text
AttorneyAddress: address
AttorneyCity: text
AttorneyState: text
CheckBox: checkbox
State_Blank: text
EffectiveDate_Checkbox_Current: date
EffectiveDate_Checkbox_SpecificDate: date
EffectiveDate_Text_Date: date
EffectiveDate_Text_Year: date
Termination_Checkbox_SpecificDate: date
Termination_Text_Date: date
Termination_Text_Year: date
Termination_Checkbox_Revocation: checkbox
Termination_Checkbox_Incapacity: checkbox
Witness_Text_Day: date
Witness_Text_Month: date
Witness_Text_Year: date
Principal_Signature: signature
Principal_PrintedName: text
Trainee_Name: text
Trainee_Address: address

In [12]:
df_map = pd.DataFrame(label_type_map.items(), columns=["label", "field_type"])

# Save to CSV
df_map.to_csv("label_type_map.csv", index=False)


In [13]:
print(df["field_type"].value_counts())

field_type
text         49
date         22
checkbox     19
address      12
amount        6
signature     4
phone         1
email         1
Name: count, dtype: int64


In [14]:
from sklearn.utils import resample

# Separate majority and minority classes
df_majority = df[df.field_type == "text"]
df_minority = df[df.field_type != "text"]

# Downsample majority only if it's larger than minority
if len(df_majority) > len(df_minority):
    df_majority_downsampled = resample(df_majority,
                                       replace=False,
                                       n_samples=len(df_minority), # sample size equal to minority class size
                                       random_state=42)
    # Combine
    df_balanced = pd.concat([df_majority_downsampled, df_minority])
else:
    # If majority is not larger, just use the original dataframe or upsample minority
    # For now, we'll just use the original dataframe as downsampling isn't needed
    df_balanced = df.copy()

Oversampling majority class

In [15]:
from sklearn.utils import resample

df_balanced = pd.concat([
    resample(df[df["field_type"] == ft],
             replace=True,
             n_samples=df["field_type"].value_counts().max(),
             random_state=42)
    for ft in df["field_type"].unique()
])


In [16]:
df_balanced["field_type"].value_counts().to_dict()

{'text': 49,
 'date': 49,
 'address': 49,
 'amount': 49,
 'checkbox': 49,
 'signature': 49,
 'phone': 49,
 'email': 49}

Undersampling majority class

In [ ]:
min_count = df["field_type"].value_counts().min()

df_balanced = pd.concat([
    resample(df[df["field_type"] == ft],
             replace=False,
             n_samples=min_count,
             random_state=42)
    for ft in df["field_type"].unique()
])

In [25]:
df_balanced["field_type"].value_counts()

,count
field_type,
text,49
date,49
address,49
amount,49
checkbox,49
signature,49
phone,49
email,49


In [ ]:
!pip install faker

In [ ]:
import faker
fake = faker.Faker()

synthetic = []
for _ in range(50):  # generate 50 dummy rows
    synthetic.append({"text": fake.phone_number(), "field_type": "phone"})
    synthetic.append({"text": fake.email(), "field_type": "email"})
    synthetic.append({"text": fake.date(), "field_type": "date"})

df_synth = pd.DataFrame(synthetic)
df_augmented = pd.concat([df, df_synth], ignore_index=True)

In [ ]:
df_augmented["label"].value_counts()

,count
label,
FreeText,20
CheckBox,17
"AdultCount, ChildCount",2
"StartDate, EndDate",2
PrincipalName,2
...,...
Default_NoticePeriod_NonPayment,1
Default_NoticePeriod_Payment,1
GoverningLaw_State,1


Test Dataset

In [17]:
download_file("https://storage.googleapis.com/legal_docs_scribd/test.docx","test.docx")

In [18]:
all_blanks = extract_blanks_with_context("test.docx")
export_to_csv(all_blanks)

Saved 25 placeholders to placeholders_20251103_035213.csv


Take Test data in csv format directly

In [19]:
download_file("https://storage.googleapis.com/legal_docs_scribd/test_csv.csv","test.csv")

In [20]:
df_test = pd.read_csv("test.csv")

In [21]:
df_test["input_text"] = df_test["context"].fillna("") + " " + df_test["text"].fillna("")

In [22]:
!pip install transformers datasets scikit-learn

In [23]:
import pandas as pd

# df = pd.read_csv("placeholder.csv")
df = df_balanced
df["text"] = df["context"].fillna("") + " " + df["text"].fillna("")

# Encode labels
label2id = {label: i for i, label in enumerate(df["field_type"].unique())}
id2label = {i: label for label, i in label2id.items()}
df["label_id"] = df["field_type"].map(label2id)

In [24]:
print(df["field_type"].value_counts())

field_type
text         49
date         49
address      49
amount       49
checkbox     49
signature    49
phone        49
email        49
Name: count, dtype: int64


In [25]:
from datasets import Dataset
from transformers import AutoTokenizer

dataset = Dataset.from_pandas(df[["text", "label_id"]].rename(columns={"label_id": "labels"})) # Renamed label_id to labels
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

dataset = dataset.map(tokenize, batched=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/392 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch", # Changed from evaluation_strategy
    per_device_train_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
)

trainer.train()

In [ ]:
def predict_label(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    predicted_id = outputs.logits.argmax(dim=1).item()
    return id2label[predicted_id]

In [38]:
from google.colab import drive
drive.mount('/content/drive')
model.config.id2label = id2label
model.config.label2id = label2id

trainer.save_model("/content/drive/MyDrive/my_model")
tokenizer.save_pretrained("/content/drive/MyDrive/my_model")

Mounted at /content/drive


NameError: name 'model' is not defined

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_path = "/content/drive/MyDrive/my_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [ ]:
import pandas as pd

df_test = pd.read_csv("test.csv")  # should have at least a 'text' column
df_test["input_text"] = df_test["context"].fillna("") + " " + df_test["text"].fillna("")

In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

preds = classifier(df_test["input_text"].tolist(), truncation=True)
df_test["predicted_label"] = [p["field_type"] for p in preds]

In [ ]:
print(df_test[["text", "predicted_label"]].head(10))

skLearn Model

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# --- Step 1: Load your labeled CSV ---
df = pd.read_csv("placeholder.csv")  # update filename if needed

# --- Step 2: Prepare features and labels ---
# Combine context + text if context is useful
df["input_text"] = df["context"].fillna("") + " " + df["text"].fillna("")
X = df["input_text"]
y = df["label"]

# --- Step 3: Vectorize text ---
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=5000)
X_vec = vectorizer.fit_transform(X)

# --- Step 4: Train/test split ---
X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, random_state=42)

# --- Step 5: Train classifier ---
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# --- Step 6: Evaluate ---
y_pred = clf.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# --- Step 7: Predict on new snippet ---
def predict_label(snippet, context=""):
    full_text = context + " " + snippet
    vec = vectorizer.transform([full_text])
    return clf.predict(vec)[0]

# Example
print(predict_label("Tenant shall pay a monthly rent of __________"))

Classification Report:
                              precision    recall  f1-score   support

             AttorneyAddress       0.00      0.00      0.00         1
                    CheckBox       0.14      0.67      0.24         3
       Company_Rep_Signature       0.00      0.00      0.00         1
                        Date       0.00      0.00      0.00         1
Default_NoticePeriod_Payment       0.00      0.00      0.00         1
     EffectiveDate_Text_Date       0.00      0.00      0.00         1
                    FreeText       0.44      1.00      0.62         4
       Guarantor_Address_Zip       0.00      0.00      0.00         1
                 MonthlyRent       0.00      0.00      0.00         1
       Premises_Address_City       0.00      0.00      0.00         1
      Premises_Address_State       0.00      0.00      0.00         1
              RegistryNumber       0.00      0.00      0.00         1
                 RentDueDate       0.00      0.00      0.00       

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
clf = LogisticRegression(max_iter=1000, class_weight="balanced")

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


In [ ]:
print(predict_label("Tenant shall pay a monthly rent of __________"))